# Building Your Predictor

The next step after preparing and importing your data via `Getting_Data_Ready.ipynb` is to build your first model.

The overall process for this is:

* Setup
* Create a Predictor
* Deploy a Predictor
* Obtain a Forecast

To get started, simply execute the cells below:


## Setup


Import the standard Python Libraries that are used in this lesson.

In [5]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time

The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [6]:
session = boto3.Session(region_name='us-east-1') 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Create a Predictor

Now in the previous notebook, your data was imported to be used by Forecast, here we will once again define your dataset information and then start building your model or predictor.

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [9]:
project = 'electric_power_forecastdemo' # This should be the same as your previous notebook

In [10]:
predictorName= project+'_prophet_algo'

In [11]:
forecastHorizon = 24

In [12]:
algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'

In [13]:
datasetGroupArn = "arn:aws:forecast:us-east-1:325928439752:dataset-group/util_power_forecastdemo_dsg" # Fill in the quotes from the output of the previous notebook.

In [8]:
create_predictor_response=forecast.create_predictor(PredictorName=predictorName, 
                                                  AlgorithmArn=algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                                  FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

In [9]:
predictorArn=create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
while True:
    predictorStatus = forecast.describe_predictor(PredictorArn=predictorArn)['Status']
    print(predictorStatus)
    if predictorStatus != 'ACTIVE' and predictorStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS


### Get Error Metrics

In [3]:
#predictorArn = "arn:aws:forecast:us-east-1:325928439752:predictor/electric_power_forecastdemo_prophet_algo"

In [7]:
forecast.get_accuracy_metrics(PredictorArn=predictorArn)

{'PredictorEvaluationResults': [{'TestWindows': [{'EvaluationType': 'SUMMARY',
     'Metrics': {'RMSE': 12.829060757375116,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.07417153979298859},
       {'Quantile': 0.5, 'LossValue': 0.1623208455868377},
       {'Quantile': 0.1, 'LossValue': 0.05899356936870511}]}},
    {'TestWindowStart': datetime.datetime(2014, 10, 30, 0, 0, tzinfo=tzlocal()),
     'TestWindowEnd': datetime.datetime(2014, 10, 31, 0, 0, tzinfo=tzlocal()),
     'ItemCount': 3,
     'EvaluationType': 'COMPUTED',
     'Metrics': {'RMSE': 12.829060757375116,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.07417153979298859},
       {'Quantile': 0.5, 'LossValue': 0.1623208455868377},
       {'Quantile': 0.1, 'LossValue': 0.05899356936870511}]}}]}],
 'ResponseMetadata': {'RequestId': '69bb5844-8c2e-46a3-b852-d70b9e33ad62',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue,

## Create a Forecast

Now create a forecast using the model that was trained

In [14]:
forecastName= project+'_prophet_algo_forecast'

In [15]:
create_forecast_response=forecast.create_forecast(ForecastName=forecastName,
                                                  PredictorArn=predictorArn)
forecastArn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [16]:
while True:
    forecastStatus = forecast.describe_forecast(ForecastArn=forecastArn)['Status']
    print(forecastStatus)
    if forecastStatus != 'ACTIVE' and forecastStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE


### Get Forecast

Once created, the forecast results are ready and you view them. 

In [17]:
print(forecastArn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecastArn,
    Filters={"item_id":"client_12"}
)
print(forecastResponse)

arn:aws:forecast:us-east-1:325928439752:forecast/electric_power_forecastdemo_prophet_algo_forecast

{'Forecast': {'Predictions': {'p10': [{'Timestamp': '2014-10-31T00:00:00', 'Value': 36.22185516357422}, {'Timestamp': '2014-10-31T01:00:00', 'Value': 34.557891845703125}, {'Timestamp': '2014-10-31T02:00:00', 'Value': 51.03107833862305}, {'Timestamp': '2014-10-31T03:00:00', 'Value': 48.43934631347656}, {'Timestamp': '2014-10-31T04:00:00', 'Value': 42.049537658691406}, {'Timestamp': '2014-10-31T05:00:00', 'Value': 33.205116271972656}, {'Timestamp': '2014-10-31T06:00:00', 'Value': 31.351680755615234}, {'Timestamp': '2014-10-31T07:00:00', 'Value': 44.99196243286133}, {'Timestamp': '2014-10-31T08:00:00', 'Value': 74.00486755371094}, {'Timestamp': '2014-10-31T09:00:00', 'Value': 107.83564758300781}, {'Timestamp': '2014-10-31T10:00:00', 'Value': 130.82528686523438}, {'Timestamp': '2014-10-31T11:00:00', 'Value': 130.74481201171875}, {'Timestamp': '2014-10-31T12:00:00', 'Value': 122.9478836059570

## Next Steps

Now that your forecast has been created, use the arn that was printed above to evaluate it in `Evaluating_Your_Predictor.ipynb`